In [1]:
import pickle

from synbio.annotations import Location, Part

import z3
from z3helpers.definitions import *
from z3helpers.constraints import *
from z3helpers.utils import add_constraints

# phiX174 specific stuffs
with open("phiX174.pkl", "rb") as handle:
    phiX174 = pickle.load(handle)
    
region = Location(1155, 1521, "FWD")
offset = region.start

geneA = Part(seq=phiX174, name="A gene", kind="CDS", location=region)
geneB = phiX174.annotations["B gene"]

# define z3 variables
T = code_dict(triplet_dna_codons, AminoBitVecSort)
dna_seq = dna_variables(region, NucleotideEnumSort)
geneA_prot_seq = protein_variables(geneA, AminoBitVecSort)
geneB_prot_seq = protein_variables(geneB, AminoBitVecSort)

# add translation constraints
geneA_translation_constraints = translation_constraints(
    T, dna_seq, geneA_prot_seq, geneA.location, aminos=z3_bitvec_aminos, offset=offset)
geneB_translation_constraints = translation_constraints(
    T, dna_seq, geneB_prot_seq, geneB.location, 
    start_flag=True, stop_flag=True, aminos=z3_bitvec_aminos, offset=offset)

# add soft constraints
translation_obj = translates_same(geneA_prot_seq, geneA, amino_to_z3_bitvec_amino)
translation_obj += translates_same(geneB_prot_seq, geneB, amino_to_z3_bitvec_amino)

In [12]:
# set up z3
z3.set_param("verbose", 10)
z3.set_param("smt.bv.eq_axioms", False)
z3.set_param("smt.phase_caching_on", 80000)
solver = z3.Optimize()
# add hard constraints
add_constraints(solver, amino_bitvec_unary_restriction())
add_constraints(
    solver, RED20(T, aminos=z3_bitvec_aminos, amino_dict=amino_to_z3_bitvec_amino)
)
add_constraints(solver, geneA_translation_constraints)
add_constraints(solver, geneB_translation_constraints)
# add soft constraints
add_constraints(solver, translation_obj, hard=False)
# solve
if solver.check() == z3.sat:
    print(solver.model())
else:
    print(z3.unsat)

unsat


# Some multiplexing
Goal: probe problem complexity by testing subsets of the problem, i.e., recoding the first $N$ codons of gene B embedded in gene A.

In [11]:
# import pickle
from typing import Optional
import pickle

from synbio.annotations import Location, Part
from synbio.polymers import DNA

import z3
from z3helpers.definitions import *
from z3helpers.constraints import *
from z3helpers.utils import add_constraints

# phiX174 specific stuffs
with open("phiX174.pkl", "rb") as handle:
    phiX174 = pickle.load(handle)
    
NUM_CODONS = 30
A_START = 1155
A_END = 1521
region = Location(A_START, A_END, "FWD")

geneA = Part(seq=phiX174, name="A gene", kind="CDS", location=region)
geneB = phiX174.annotations['B gene']

def get_part(base_part: Part, num_codons: int, start: Optional[int] = None) -> Part:
    base_loc = base_part.location
    base_seq = base_part._seq_reference
    base_name = base_part.name
    base_kind = base_part.kind
    
    if start is None:
        start = base_loc.start
    
    new_loc = Location(start, int(start + 3*num_codons), "FWD")
    return Part(seq=base_seq, name=f"{base_name} ({num_codons} codons)", 
                kind=base_kind, location=new_loc)

subset_A = get_part(geneA, NUM_CODONS, start=A_START)
subset_B = get_part(geneB, NUM_CODONS)

# define z3 variables
T = code_dict(triplet_dna_codons)
dna_seq = dna_variables(region)
geneA_prot_seq = protein_variables(subset_A)
geneB_prot_seq = protein_variables(subset_B)

# add translation constraints
geneA_translation_constraints = translation_constraints(
    T, dna_seq, geneA_prot_seq, subset_A.location, offset=A_START)
geneB_translation_constraints = translation_constraints(
    T, dna_seq, geneB_prot_seq, subset_B.location, 
    start_flag=True, offset=A_START)

# add soft constraints
translation_obj = translates_same(geneA_prot_seq, subset_A)
translation_obj += translates_same(geneB_prot_seq, subset_B)

# set up z3
z3.set_param("verbose", 10)
z3.set_param("smt.bv.eq_axioms", False)
z3.set_param("smt.phase_caching_on", 80000)
solver = z3.Optimize()
solver.set("timeout", 1000)
# add hard constraints
add_constraints(solver, RED20(T))
add_constraints(solver, geneA_translation_constraints)
add_constraints(solver, geneB_translation_constraints)
# add soft constraints
add_constraints(solver, translation_obj, hard=False)
# solve
solver.check()

unknown

ctrl_c (bool) enable interrupts from ctrl-c (default: true)
dump_benchmarks (bool) dump benchmarks for profiling (default: false)
dump_models (bool) display intermediary models to stdout (default: false)
elim_01 (bool) eliminate 01 variables (default: true)
enable_sat (bool) enable the new SAT core for propositional constraints (default: true)
enable_sls (bool) enable SLS tuning during weighted maxsast (default: false)
maxlex.enable (bool) enable maxlex heuristic for lexicographic MaxSAT problems (default: true)
maxres.add_upper_bound_block (bool) restict upper bound with constraint (default: false)
maxres.hill_climb (bool) give preference for large weight cores (default: true)
maxres.max_core_size (unsigned int) break batch of generated cores if size reaches this number (default: 3)
maxres.max_correction_set_size (unsigned int) allow generating correction set constraints up to maximal size (default: 3)
maxres.max_num_cores (unsigned int) maximal number of cores per round (default: 429

In [2]:
from multiprocessing import Pool, Queue, Process
from collections import namedtuple
from typing import NewType
import time

# NUMBER_OF_PROCESSES = 8
# Job = namedtuple("Job", "fxn args".split())
# Result = namedtuple("Result", "a b c".split())

# job_queue = Queue()
# result_queue = Queue()

def sleepy(timeout=0.1):
    print("gonna take a nap")
    time.sleep(timeout)
    print("mmmm, what a nice nap")

def killjoy(timeout=3):
    pass

